In [7]:
#Pre-process the Reviews
import random
import sys
import random
import re
from collections import defaultdict

ifile1 = open("full-meta-data.txt")
revid = 0
users = defaultdict(list)
for ln in ifile1:
    parts = ln.strip().split("\t")
    users[parts[0]].append(revid)
    revid = revid + 1
ifile1.close()
userids = []
#print(users)
c =0
#Select reviewer subset based on tunable parameters (max-reviews and min-reviews limit,sampling ratio)
minreviews =10
maxreviews = 25
samplingratio = 0.02
for u in users:
    if len(users[u])>minreviews and len(users[u])<maxreviews:
        if random.random() < samplingratio:
            userids.append(u)
            c= c + len(users[u])
print("Total Reviews="+str(c))



ifile = open("reviewContent.txt",encoding='ISO-8859-1')
ifile1 = open("full-meta-data.txt")
s_words = []
sfile = open("Words.txt")
for ln in sfile:
    s_words.append(ln.strip())
sfile.close()
stopwords = []
sfile = open("stopwords.txt")
for ln in sfile:
    stopwords.append(ln.strip())
sfile.close()

flags = (re.UNICODE if sys.version < '3' and type(text) is unicode
         else 0)
ofile = open("all_revs1.txt",'w')
ofile1 = open("metadata.txt",'w')
cnt = 0
revid = 0
windex = defaultdict(list)
#Tunable parameter to keep non-sentiment words
PNonSentWords = 0.25

for ln in ifile:
    ln1 = ifile1.readline()
    parts1 = ln1.strip().split("\t")
    #print(parts1)
    if parts1[0] not in userids:
        continue
    #if cnt >= 10000:
    #    break
    keep = []
    parts = ln.strip().split("\t")
    print(parts[0])
    for word in re.findall(r"\w[\w']*", parts[3], flags=flags):
        if word.isdigit() or len(word)==1:
            continue
        word_lower = word.lower()
        if word_lower in stopwords:
            continue
        if word_lower in s_words:
            keep.append(word_lower)
        elif random.random() < PNonSentWords:
            keep.append(word_lower)
    if float(parts1[2]) <3:
        cl = 0
    elif float(parts1[2]) == 3:
        cl = 1
    else:
        cl = 2
    if len(keep)>10:
        cnt = cnt + 1
        ofile.write(" ".join(keep)+"\t"+str(cl)+"\n")
        ofile1.write(ln1)
        for w in keep:
            windex[w].append(revid)
        revid = revid + 1
ofile.close()
ofile1.close()
ifile.close()
ifile1.close()

#Tunable parameter (keep words only if repeated in > NumReps reviews)
NumReps = 10

#Filter review words
ifile = open("all_revs1.txt",encoding="ISO-8859-1")
ofile = open("processed_revs_11.txt",'w')
for ln in ifile:
    parts = ln.strip().split("\t")
    keep = []
    for w in parts[0].split(" "):
        if len(windex[w])>NumReps:
            keep.append(w)
    ofile.write(" ".join(keep)+"\t"+parts[1]+"\n")
ofile.close()
ifile.close()

Total Reviews=1772
5375
5419
6414
7454
8754
8981
9548
10032
10269
10290
10419
8981
11603
11770
11984
12432
5375
12609
12667
13008
13863
13877
14269
14488
14535
11603
14852
14879
15210
14879
11984
15987
16851
16937
16941
17118
17286
17367
17960
18231
10269
14269
19263
19910
12667
21216
21758
8754
12432
23691
24025
23691
24522
7454
24661
25505
26079
24522
10269
11984
26713
26742
26981
27630
28063
15210
8754
11603
18231
28348
9548
28831
28983
24522
29335
29353
29335
29353
30465
30792
30828
31232
31644
31838
31845
32719
26713
33480
32719
32719
34532
10419
10419
35096
11603
33480
14879
28348
35893
12667
31845
36456
24522
36528
36560
37311
37316
35096
37754
26981
16851
26079
39637
14535
8754
7454
6414
13863
41208
41387
37754
41601
41208
6414
28983
43579
14535
28063
10419
43579
44409
26079
17367
41208
31232
34532
33480
16941
46426
46710
46762
36528
24025
47025
47112
13008
47212
33480
48628
5375
17367
9548
49009
49171
28348
17367
12609
49805
29353
30792
10269
25505
11770
41601
50761
51037
3731

26713
48628
46426
24661
17286
41601
17960
17367
16851
99876
8754
34532
30465
11770
18231
41208
70012
25505
8754
12432
41387
9548
12609
17118
24522
12667
17367
98448
49805
44409
21758
69724
48628
28831
61457
18231
51037
49805
10419
24661
14269
49805
44409
51700
37311
50761
35096
16851
88597
12609
28983
35893
24661
15987
12432
87777
23691
67100
30828
16851
49171
13877
49805
43579
88597
15210
21758
31232
16937
13863
5419
14879
26742
70012
33480
18231
10290
19910
46426
25505
12667
35096
30828
24522
51037
31845
46762
67100
28983
35893
98448
14488
13863
48628
46426
28063
5419
47212
41208
12609
28831
34532
30465
35096
26981
41601
10269
46710
48628
9548
15210
26079
70012
10419
50761
69724
13008
47025
8754
69724
47212
37311
69724
11603
88597
47212
10032
49805
51700
14852
30828
47025
32719
34532
15210
13008
26079
14852
29335
98448
69904
7454
14535
11770
8754
69724
51700
51700
24661
5419
17286
31845
51700
15210
28063
69724
19263
28983
43579
10419
43579
99876
48628
31845
5375
25505
88597
67100
868

In [2]:
#SVM Learner
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
#from sklearn import cross_validation
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler

#Learn SVM Model
ifile = open("processed_revs_1.txt",encoding="ISO-8859-1")
Y = []
words = []
unique_words = []
for ln in ifile:
    parts = ln.strip().split("\t")
    Y.append(int(parts[1]))
    words.append(parts[0])
    for w in parts[0].split():
        if w not in unique_words:
            unique_words.append(w)
ifile.close()
tf_transformer = TfidfVectorizer()
f = tf_transformer.fit_transform(words)
features = [((i, j), f[i,j]) for i, j in zip(*f.nonzero())]
unique_word_ids = []
for w in unique_words:
    i = tf_transformer.vocabulary_.get(w)
    unique_word_ids.append(i)

clf = svm.LinearSVC(C=1)
clf.fit(f,Y)

#Store learned weights

#Tunable parameter, normalization range of weights
rangelower = -1
rangehigher = 1

C = clf.coef_
scaler = MinMaxScaler(feature_range=(rangelower,rangehigher))
vals = []
for i, j in zip(*C.nonzero()):
    vals.append([C[i,j]])
scaler.fit(vals)
V1 = scaler.transform(vals)
rows,cols = C.nonzero()
r_wts =  defaultdict(list)
ix= 0
for w in unique_words:
    r_wts[w] = [0,0,0]    
for i, j in zip(*C.nonzero()):
    for k in tf_transformer.vocabulary_.keys():
        if tf_transformer.vocabulary_[k]==j:
            if k not in r_wts:
                break
            else:
                r_wts[k][i] = V1[ix][0]
                break
    ix = ix + 1


print(r_wts)

defaultdict(<class 'list'>, {'liked': [0.23597472220104448, 0.3730159967840451, -0.5256804851193937], 'good': [-0.2667244948122437, 0.34801904135507833, -0.2191832669095308], 'cute': [0.17409865928509208, -0.11907571279704376, -0.09520205458549082], 'authentic': [0.023647351198159604, 0.24229167712917654, -0.17644875110326694], 'disliked': [0.20066237977245185, 0.29162147856230497, -0.537731385692079], 'loud': [0.051538591689547164, -0.009474257012038682, -0.03152155125765513], 'chicken': [-0.4325884585910796, 0.4108622004415303, -0.13967397692358408], 'love': [-0.467842587648547, -0.1049116437980851, 0.35568660925302803], 'place': [0.3824175330445036, -0.10436761761480477, -0.22282116596417884], 'go': [-0.02656449334692184, 0.05504496355390799, -0.018133714219153038], 'back': [0.07430464756296869, 0.06587346275881636, -0.06595701860438709], 'night': [-0.23448641964085493, 0.5295416813425151, -0.284217399377739], 'recommend': [0.13153513417780466, -0.5207011100162207, 0.390899537391841

In [3]:
#Store similarity relations based on if they were written by same user
import sys
import numpy as np
import collections
ifile = open("processed_revs_1.txt",encoding="ISO-8859-1")
ifile1 = open("metadata.txt")
SIM = collections.defaultdict(list)
userindex = {}
revid = 0
for ln in ifile:
    parts = ln.strip().split("\t")
   # print(parts)
    ln1 = ifile1.readline()
    parts1 = ln1.strip().split("\t")
   # print(parts1)
    SIM[parts1[0]].append(revid)
    userindex[revid] = parts1[0]
    revid = revid + 1    
ifile.close()
ifile1.close()
#print(userindex)
#sys.exit()
sameusers = []
gh=open("filesameuserlist.txt","w")
for s in SIM.keys():
    for i in range(0,len(SIM[s])-1,1):
        gg="Sameuser("+str(SIM[s][i])+","+str(SIM[s][i+1])+")"
        sameusers.append(gg)
ddf=open("Sameusert.txt","w")
for t in sameusers:
    ddf.write(str(t)+"\n")
#Compute Similarity weights for a word
#Tunable parameter threshold
s_treshold = 0.01
SIM_wts = {}
SVec = []
wordslt = []
#print(windex)
for w in windex.keys():
    lst = []
    try:
        for r in windex[w]:
            lst.append(int(userindex[r]))
        h = np.histogram(lst)
        SVec.append([h[0].mean()])
        wordslt.append(w)
    except:
        continue
#print(SVec)
#sys.exit()
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(SVec)
SVec1 = scaler.transform(SVec)
for i,w in enumerate(wordslt):
    if SVec1[i][0] > s_treshold:
        SIM_wts[w] = SVec1[i][0]
print("hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh")
#print(SIM_wts)
opf=open("sim_w.txt","w")
for t in SIM_wts:
    ff=str(SIM_wts[t])+":"+str(t)
    opf.write(str(ff)+"\n")
dfff=open("similarformulas.txt","w")
for qqq in SIM_wts:
    if float(SIM_wts[qqq])>=1.0:
        hhh="1.0"+" "+"!Has_"+str(qqq)+"(v0)"+" v "+"!Sameuser(v0,v1)"+" v "+"Has_"+str(qqq)+"(v1)"
        dfff.write(str(hhh)+"\n")
    else:
        hhh=str(SIM_wts[qqq])+" "+"!Has_"+str(qqq)+"(v0)"+" v "+"!Sameuser(v0,v1)"+" v "+"Has_"+str(qqq)+"(v1)"
        dfff.write(str(hhh)+"\n")

        

hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh


In [4]:
#Load the data structures for the sampler

trueratings = {}
WORDS = defaultdict(list)
word_index = defaultdict(list)

#Tunable parameter (what percentage of words should be visible)
hidden_words_ratio = 0.25
R1 = 0.5

ifile = open("processed_revs_1.txt")
revid = 0
S_RATING = {}
S_WORDS = defaultdict(list) 
Hidden = defaultdict(list)
Q = defaultdict(list)

revids = []
for ln in ifile:
    parts = ln.strip().split("\t")
    trueratings[revid] = parts[1]
    words = parts[0].split()
    unique = set(words)
    words = list(unique)
    for w in words:
        if w in s_words:
            if random.random() < R1:
                word_index[w].append(revid)
                WORDS[revid].append(w)
            else:
                Hidden[revid].append(w)
                Q[revid].append(0)
                S_WORDS[revid].append(random.randint(0,1))
        else:
            if random.random() < hidden_words_ratio:
                Hidden[revid].append(w)
                Q[revid].append(0)
                S_WORDS[revid].append(random.randint(0,1))
            else:
                word_index[w].append(revid)
                WORDS[revid].append(w)
                
    revids.append(revid)
    S_RATING[revid] = random.randint(0,2)
    revid = revid + 1
print(WORDS)
fk=open("filewordrevmap.txt","w")
for q in WORDS:
    hh=str(q)+":"+str(WORDS[q])
    fk.write(str(hh)+"\n")
ifile.close()





defaultdict(<class 'list'>, {0: ['cute', 'chicken', 'back', 'go', 'place', 'disliked', 'good', 'recommend'], 1: ['another', 'spectacular', 'came', 'reservations', 'good'], 2: ['also', 'people', 'along', 'eat', 'disappointment', 'worst', 'pretty', 'places'], 3: ['give', 'best', 'think'], 4: ['lot', 'enjoy', 'steak', 'table', 'food', 'gross', 'authentic', 'also', 'busy', 'poor', 'meat', 'things', 'cheese', 'review', 'feeling', 'place', 'fatty'], 5: ['especially', 'way', 'sandwich', 'getting'], 6: ['liked', 'slow', 'ridiculously', 'best', 'will', 'location', 'went', 'things', 'disappointment'], 7: ['nice', 'pancakes', 'helpful', 'yes', 'time', 'sour', 'disappoint', 'one', 'love', 'wait', 'eggs', 'cool', 'place'], 8: ['well', 'side', 'food', 'best', 'let', 'decor', 'especially', 'back', 'look', 'say', 'better', 'impressed', 'end', 'different', 'times', 'busy', 'see', 'compliment', 'think', 'though', 'bland'], 9: ['come', 'price', 'right', 'service', 'good', 'however', 'flavor', 'clean', 'p

In [5]:
import random
from collections import defaultdict

#Sampler

#Tunable parameter (default value of prob)
C1 = 0.98
iters = 1000
for t in range(0,iters,1):
    r1 = random.randint(0,len(revids)-1)
    r = revids[r1]
    if random.random()<0.5:
        #sample rating
        W0 = 0
        W1 = 0
        W2 = 0
        for w in WORDS[r]:
            if len(r_wts[w])<3:
                continue
            W0 = W0 + r_wts[w][0]
            W1 = W1 + r_wts[w][1]
            W2 = W2 + r_wts[w][2]
        if r in Hidden:
            for j,w in enumerate(Hidden[r]):
                if len(r_wts[w])<3:
                    continue

                if S_WORDS[r][j]==1:
                    W0 = W0 + r_wts[w][0]
                    W1 = W1 + r_wts[w][1]
                    W2 = W2 + r_wts[w][2]
        if (W0+W1+W2) != 0:
            W0 = W0/(W0+W1+W2)
            W1 = W1/(W0+W1+W2)
            W2 = W2/(W0+W1+W2)
            sval = random.random()
            if sval<W0:
                S_RATING[r]=0
            elif sval<(W0+W1):
                S_RATING[r]=1
            else:
                S_RATING[r]=2
    else:
        #Sample words
        if r not in Hidden:
            continue
        for i,w in enumerate(Hidden[r]):
            if len(r_wts[w])<3:
                continue
            W0 = 0
            W1 = 0
            W1 = W1 + r_wts[w][S_RATING[r]]
            W0 = W0 + C1
            U = userindex[r]
            for rev1 in SIM[U]:
                if rev1 == r:
                    continue
                for e1 in WORDS[rev1]:
                    if e1==w and e1 in SIM_wts:
                        W1 = W1 + SIM_wts[e1]
                        W0 = W0 + C1
                if rev1 in Hidden:
                    for k,e1 in enumerate(Hidden[rev1]):
                        if e1==w and S_WORDS[rev1][k]==1 and e1 in SIM_wts:
                            W1 = W1 + SIM_wts[e1]
                            W0 = W0 + C1
            if W0+W1!=0:
                W0 = W0/(W0+W1)
                W1= W1/(W0+W1)
                if random.random()<W0:
                    S_WORDS[r][i]=0
                else:
                    S_WORDS[r][i]=1
    z = 0
    p = 0
    if r not in S_WORDS:
        continue
    for j, s in enumerate(S_WORDS[r]):
        if len(r_wts[Hidden[r][j]])<3:
            continue
        if s==1:
            Q[r][j] = Q[r][j] + r_wts[Hidden[r][j]][S_RATING[r]]
print(S_RATING)

{0: 2, 1: 0, 2: 2, 3: 2, 4: 1, 5: 0, 6: 0, 7: 0, 8: 2, 9: 0, 10: 1, 11: 2, 12: 1, 13: 1, 14: 1, 15: 0, 16: 0, 17: 2, 18: 0, 19: 0, 20: 1, 21: 0, 22: 1, 23: 2, 24: 0, 25: 0, 26: 0, 27: 2, 28: 2, 29: 2, 30: 0, 31: 2, 32: 1, 33: 0, 34: 0, 35: 2, 36: 2, 37: 0, 38: 0, 39: 0, 40: 2, 41: 1, 42: 2, 43: 1, 44: 0, 45: 0, 46: 1, 47: 0, 48: 0, 49: 1, 50: 1, 51: 1, 52: 2, 53: 1, 54: 0, 55: 0, 56: 2, 57: 0, 58: 0, 59: 0, 60: 0, 61: 1, 62: 0, 63: 0, 64: 2, 65: 1, 66: 0, 67: 0, 68: 0, 69: 2, 70: 0, 71: 0, 72: 2, 73: 2, 74: 1, 75: 0, 76: 1, 77: 1, 78: 1, 79: 0, 80: 1, 81: 1, 82: 0, 83: 2, 84: 0, 85: 2, 86: 0, 87: 0, 88: 0, 89: 2, 90: 0, 91: 1, 92: 1, 93: 1, 94: 1, 95: 2, 96: 2, 97: 0, 98: 2, 99: 0, 100: 2, 101: 2, 102: 0, 103: 2, 104: 2, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 2, 112: 2, 113: 1, 114: 2, 115: 2, 116: 0, 117: 1, 118: 0, 119: 0, 120: 2, 121: 1, 122: 0, 123: 0, 124: 1, 125: 0, 126: 0, 127: 2, 128: 2, 129: 0, 130: 2, 131: 0, 132: 1, 133: 1, 134: 0, 135: 2, 136: 2, 137: 0, 138: 

In [6]:
#Generate explanations
ofile = open("expns.txt",'w')
#print(Q)
for q in Q.keys():
    L = Q[q]
    exp = []
    expvals = []
    for i,l in enumerate(L):
        if l > 0:
            exp.append(Hidden[q][i])
            expvals.append(l)
    A = np.argsort(expvals)
    vexp = []
    for a in A:
        vexp.append(exp[a])
        if len(vexp) >= 5:
            break
    if len(vexp)==5:
        ofile.write(str(q)+" "+str(trueratings[q])+" "+",".join(vexp)+":"+"\n")
ofile.close()